### Prelim paper 2021

**Q1.** The file `covid19.csv` contains the confirmed COVID-19 cases globally over a specified period.

# Task 1.1
Generate the following meta-data


Number of Countries(86), Number of days recorded (43)


In [1]:
import csv
f=open("covid19.csv","r")
data = list(csv.reader(f))
countries = set([ row[1] for row in data[1:] ]) # 2m,get a unique set of countries 
len(countries), len(data[0][4:]), data[0][4], data[0][-1] # 1m 

#1.1 [3m]


(86, 43, '1/22/20', '3/4/20')

# Task 1.2 :
Group By countries and aggregate the cases for each date
Example output:

Country,        1/22,1/23,1/24...

Mainland China, 547,639,916,

South Korea,    1,1,2,2, 

Iran,           0,0,0,0, 
             


In [2]:

import csv
f=open("covid19.csv","r")
data = list(csv.reader(f))
header = data[0]
data = sorted(data[1:], key=lambda x: x[1])
stats=[]

## 1m seperate header and data
## 2m for sorting based on country or any iteration to extract a set of countries
## [3]

In [3]:
## This solution perform only 1 pass through the sorted data 
## takes the first row,first col of the country to tally
## loop till row is of a different country
## update tally in stats 
## truncate off the country that has been tally
## repeat until the list is empty
while data:
    tally=[0]*(len(header)-4) ## 1m init tally  for each country
    i = 0
    country = data[0][1] 
    while i < len(data) and data[i][1] == country: #aggregate 1m
        for col in range(len(tally)): ## iterate for all days 1m
            tally[col] += int(data[i][4+col]) # 
        i+=1
    tally  = [country] + tally # total for each country 1m
    stats.append(tally)
    data = data[i:] # slice off rows from same country 1m
    
##[5]

In [4]:
## 
f=open("countries.csv", "w")
f.write(f"{header[1]},{','.join(header[4:])}\n")
for row in stats: # 1m iterate over counters
    s=','.join([str(c) for c in row]) # 1m convert to string and write
    f.write(s+"\n") 
f.close()

## [2m]

## 1.2 [10m]

## Alternative solution is to use a outer loop for each country
#  and a inner loop to aggregate the data for each country
#  if a dictionary is used, you still need code to "sort" the dictionary


# Task 1.3
Determine the top five country/region and province/state (if applicable) with the largest single day decrease in confirmed cases and the date this occurred. [5]

Example output:
```
Mainland China (Hubei) X 2020-02-25
```

In [5]:

import csv
tally=[]  # container to hold max for each country 1m
f=open("countries.csv","r")
cvs_reader = csv.reader(f)
rows = [r for r in cvs_reader]
header =rows[0]
## tally for each country
for r in rows[1:]: ##for each country, 1m skip header
    max_incr=0
    max_day=""
    for col in range(2,len(r)): # for each month, find max 2m
        m=int(r[col])-int(r[col-1])
        if m > max_incr:
            max_day = header[col]
            max_incr = m
    tally.append((r[0],max_incr,max_day))

top=sorted(tally, reverse=True, key=lambda x: x[1]) # get top 5 1m
for i in range(5):
    print(f"{top[i][0]:<20}{top[i][1]:<8}{top[i][2]:>10}")
    


Mainland China      15133      2/13/20
South Korea         851         3/3/20
Iran                835         3/3/20
Italy               587         3/4/20
Others              99         2/17/20


# Task 1.4

In [ ]:
## {date:"1/1/20",[{country:"china",cases:10}, ... ]}
import pymongo
client = pymongo.MongoClient("127.0.0.1", 27017)
db = client.get_database("covid")
coll = db.get_collection("cases") #1m for db connection
for i,day in enumerate(rows[0][1:]): # 1m outer loop for days
    doc = {}
    doc["date"]=day
    country_list=[] 
    for row in rows[1:]: # 1m inner loop for countries array
        country={}
        country["country"] = row[0] # 1m for correct attribues
        country["cases"] = int(row[i+1])
        country_list.append(country)
    doc["countries"] = country_list 
    coll.insert_one(doc) #1m insert 
client.close()

## [5m]

# Task 1.5

In [ ]:
## Using projection to extract the element in array
import pymongo
client = pymongo.MongoClient("127.0.0.1", 27017)
db = client.get_database("covid")
coll = db.get_collection("cases")
result=coll.find_one(
    {"date":"1/28/20", "countries.country":"Mainland China"},#1m
    {"_id":0, "countries.$":1} #1m
)
print("Cases on 1/28/20 is",result["countries"][0]["cases"])

In [ ]:
## Using code to iterate through the array and print only the elment
import pymongo
client = pymongo.MongoClient("127.0.0.1", 27017)
db = client.get_database("covid")
coll = db.get_collection("cases")
result=coll.find_one(
    {"date":"1/28/20"},
) #1m
for country in result["countries"]: #1m
    if country["country"] == "Mainland China":
        print("Cases on 1/28/20 is",country["cases"])

# Grace's solution using dictionary 
# How to sort the dictionary

In [ ]:
# Task 1.2
import csv
countries = {}

with open("COVID19.CSV") as csv_file:
    csv_reader = csv.reader(csv_file)
    count = 0
    for row in csv_reader:
        if count == 0:
            heading = row
            count += 1
            continue
        
        # check if country is in countries
        if row[1] not in countries:
            countries[row[1]] = [ int(c) for c in row[4:] ]
        # if country is in countries, aggregate the data
        else:
            temp = [] # temporary list
            for i in range(4, len(row)):
                aggregate = (int(row[i]) + int(countries[row[1]][i - 4]))
                temp.append(aggregate)
            countries[row[1]] = temp

In [ ]:
dictlist= list(countries.items())
dictlist.sort( key=lambda x: x[0])
for country in dictlist:
    print( country[0], end=",")
    print( ",".join( map(str,country[1]) ) )


In [42]:
d = {"a":[1,2,3], "z":[5,6,7],"A":[2,1,2]}
l=list(d.items())
print(l)
for i in d:
    print(i, d[i])

[('a', [1, 2, 3]), ('z', [5, 6, 7]), ('A', [2, 1, 2])]
a
z
A
